In [1]:
from datetime import datetime

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import patsy


from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

from scipy.stats import ks_2samp

In [2]:
#  Abrindo o dataset MENA
df = pd.read_excel(r"C:\Users\sabri\MENATeams.xlsx")
df.head()

,replay_id,replay_title,map,date,result,team_name,opposing_team_name,score,goals,assists,saves,shots,shots_conceded,shooting_percentage,demos_inflicted,demos_taken,save_conceded
0,000f0756-1466-488f-99c3-919141289949,MENA C R1 vs TS G3 2023-02-16.15.24,DFH Stadium,2023-02-16,win,TEAM SERENE,RULE ONE,1576,3,2,10,9,14,33.333333,1,4,0.714286
1,000f0756-1466-488f-99c3-919141289949,MENA C R1 vs TS G3 2023-02-16.15.24,DFH Stadium,2023-02-16,loss,RULE ONE,TEAM SERENE,880,0,0,5,14,9,0.000000,4,1,0.555556
2,005a5a89-368c-4fa7-a3a2-407fb45e3aa7,MENA AC TWIS vs CAM G2 2023-05-05.20.23,Forbidden Temple,2023-05-05,loss,CAM,TWISTED MINDS,795,1,1,3,7,8,14.285714,2,4,0.375000
3,005a5a89-368c-4fa7-a3a2-407fb45e3aa7,MENA AC TWIS vs CAM G2 2023-05-05.20.23,Forbidden Temple,2023-05-05,win,TWISTED MINDS,CAM,985,2,1,4,8,7,25.000000,4,2,0.571429
4,00bf66bb-05e7-418e-91bb-7ed005e1729c,MENA O T2K vs VSN G4 2022-10-20.18.43,Utopia Coliseum (Dusk),2022-10-20,loss,VISION ESPORTS,T2K,946,1,0,6,4,10,25.000000,1,1,0.600000


In [3]:
#  Renomear a coluna assists para goals_assists e goals para goals_solo
df.rename(columns={'assists':'goals_assists'}, inplace = True)
df.rename(columns={'goals':'goals_solo'}, inplace = True)
df

,replay_id,replay_title,map,date,result,team_name,opposing_team_name,score,goals_solo,goals_assists,saves,shots,shots_conceded,shooting_percentage,demos_inflicted,demos_taken,save_conceded
0,000f0756-1466-488f-99c3-919141289949,MENA C R1 vs TS G3 2023-02-16.15.24,DFH Stadium,2023-02-16,win,TEAM SERENE,RULE ONE,1576,3,2,10,9,14,33.333333,1,4,0.714286
1,000f0756-1466-488f-99c3-919141289949,MENA C R1 vs TS G3 2023-02-16.15.24,DFH Stadium,2023-02-16,loss,RULE ONE,TEAM SERENE,880,0,0,5,14,9,0.000000,4,1,0.555556
2,005a5a89-368c-4fa7-a3a2-407fb45e3aa7,MENA AC TWIS vs CAM G2 2023-05-05.20.23,Forbidden Temple,2023-05-05,loss,CAM,TWISTED MINDS,795,1,1,3,7,8,14.285714,2,4,0.375000
3,005a5a89-368c-4fa7-a3a2-407fb45e3aa7,MENA AC TWIS vs CAM G2 2023-05-05.20.23,Forbidden Temple,2023-05-05,win,TWISTED MINDS,CAM,985,2,1,4,8,7,25.000000,4,2,0.571429
4,00bf66bb-05e7-418e-91bb-7ed005e1729c,MENA O T2K vs VSN G4 2022-10-20.18.43,Utopia Coliseum (Dusk),2022-10-20,loss,VISION ESPORTS,T2K,946,1,0,6,4,10,25.000000,1,1,0.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2731,ffbe48ee-16fb-44c7-ac32-09d5c61b02ce,MENA W TWIS vs COLA G5 2023-02-02.17.26,Champions Field,2023-02-02,win,COLA,TWISTED MINDS,1105,2,2,4,9,6,22.222222,3,1,0.666667
2732,ffc597d4-844a-44f5-9a44-c65fa64012e1,MENA B TWIS vs VELO G2 2022-11-04.17.18,Aquadome,2022-11-04,win,VELOCE ESPORTS,TWISTED MINDS,1397,3,1,8,10,9,30.000000,3,9,0.888889
2733,ffc597d4-844a-44f5-9a44-c65fa64012e1,MENA B TWIS vs VELO G2 2022-11-04.17.18,Aquadome,2022-11-04,loss,TWISTED MINDS,VELOCE ESPORTS,969,1,1,5,9,10,11.111111,9,3,0.500000
2734,ffe7e43a-de0d-4c76-a75a-69c6623025dc,MENA P EMP vs RX G3 2023-03-02.16.20,DFH Stadium,2023-03-02,win,EMPTY,RX,1540,4,2,7,9,10,44.444444,6,3,0.700000


In [4]:
#  Consertando os valores da variável goals solo para excluir os valores de goals_assists
df['goals_solo'] = df['goals_solo'] - df['goals_assists']
df

,replay_id,replay_title,map,date,result,team_name,opposing_team_name,score,goals_solo,goals_assists,saves,shots,shots_conceded,shooting_percentage,demos_inflicted,demos_taken,save_conceded
0,000f0756-1466-488f-99c3-919141289949,MENA C R1 vs TS G3 2023-02-16.15.24,DFH Stadium,2023-02-16,win,TEAM SERENE,RULE ONE,1576,1,2,10,9,14,33.333333,1,4,0.714286
1,000f0756-1466-488f-99c3-919141289949,MENA C R1 vs TS G3 2023-02-16.15.24,DFH Stadium,2023-02-16,loss,RULE ONE,TEAM SERENE,880,0,0,5,14,9,0.000000,4,1,0.555556
2,005a5a89-368c-4fa7-a3a2-407fb45e3aa7,MENA AC TWIS vs CAM G2 2023-05-05.20.23,Forbidden Temple,2023-05-05,loss,CAM,TWISTED MINDS,795,0,1,3,7,8,14.285714,2,4,0.375000
3,005a5a89-368c-4fa7-a3a2-407fb45e3aa7,MENA AC TWIS vs CAM G2 2023-05-05.20.23,Forbidden Temple,2023-05-05,win,TWISTED MINDS,CAM,985,1,1,4,8,7,25.000000,4,2,0.571429
4,00bf66bb-05e7-418e-91bb-7ed005e1729c,MENA O T2K vs VSN G4 2022-10-20.18.43,Utopia Coliseum (Dusk),2022-10-20,loss,VISION ESPORTS,T2K,946,1,0,6,4,10,25.000000,1,1,0.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2731,ffbe48ee-16fb-44c7-ac32-09d5c61b02ce,MENA W TWIS vs COLA G5 2023-02-02.17.26,Champions Field,2023-02-02,win,COLA,TWISTED MINDS,1105,0,2,4,9,6,22.222222,3,1,0.666667
2732,ffc597d4-844a-44f5-9a44-c65fa64012e1,MENA B TWIS vs VELO G2 2022-11-04.17.18,Aquadome,2022-11-04,win,VELOCE ESPORTS,TWISTED MINDS,1397,2,1,8,10,9,30.000000,3,9,0.888889
2733,ffc597d4-844a-44f5-9a44-c65fa64012e1,MENA B TWIS vs VELO G2 2022-11-04.17.18,Aquadome,2022-11-04,loss,TWISTED MINDS,VELOCE ESPORTS,969,0,1,5,9,10,11.111111,9,3,0.500000
2734,ffe7e43a-de0d-4c76-a75a-69c6623025dc,MENA P EMP vs RX G3 2023-03-02.16.20,DFH Stadium,2023-03-02,win,EMPTY,RX,1540,2,2,7,9,10,44.444444,6,3,0.700000


In [5]:
#  Checando se há empates nos resultados e removendo-os
df_draw = df[df['result']=='draw']
df = df.drop(df_draw.index)

In [6]:
#  Transformando result em variável binária. Loss = 1 e win = 0
df1 = pd.get_dummies(df['result'])
#df1.head()
df1 = df1.drop(['win'], axis = 1)
#df1.head()
df = pd.concat((df, df1), axis = 1)
#df.head()
df = df.drop(['result'], axis = 1)
df = df.rename(columns = {'loss': 'result'})
df.head()

,replay_id,replay_title,map,date,team_name,opposing_team_name,score,goals_solo,goals_assists,saves,shots,shots_conceded,shooting_percentage,demos_inflicted,demos_taken,save_conceded,result
0,000f0756-1466-488f-99c3-919141289949,MENA C R1 vs TS G3 2023-02-16.15.24,DFH Stadium,2023-02-16,TEAM SERENE,RULE ONE,1576,1,2,10,9,14,33.333333,1,4,0.714286,0
1,000f0756-1466-488f-99c3-919141289949,MENA C R1 vs TS G3 2023-02-16.15.24,DFH Stadium,2023-02-16,RULE ONE,TEAM SERENE,880,0,0,5,14,9,0.000000,4,1,0.555556,1
2,005a5a89-368c-4fa7-a3a2-407fb45e3aa7,MENA AC TWIS vs CAM G2 2023-05-05.20.23,Forbidden Temple,2023-05-05,CAM,TWISTED MINDS,795,0,1,3,7,8,14.285714,2,4,0.375000,1
3,005a5a89-368c-4fa7-a3a2-407fb45e3aa7,MENA AC TWIS vs CAM G2 2023-05-05.20.23,Forbidden Temple,2023-05-05,TWISTED MINDS,CAM,985,1,1,4,8,7,25.000000,4,2,0.571429,0
4,00bf66bb-05e7-418e-91bb-7ed005e1729c,MENA O T2K vs VSN G4 2022-10-20.18.43,Utopia Coliseum (Dusk),2022-10-20,VISION ESPORTS,T2K,946,1,0,6,4,10,25.000000,1,1,0.600000,1


In [7]:
#  Excluindo várias colunas de uma vez
df.drop(['replay_id','replay_title','map','team_name','opposing_team_name', 'shots_conceded'], axis = 1, inplace = True)

In [8]:
#  Apagando as observações vazias
df.dropna(axis = 0, inplace = True)

In [9]:
#  Dividindo a base de validação
periodo_des = df[df['date'] < datetime(2023,6,1)].copy()
oot = df[df['date'] > datetime(2023,6,1)].copy()
periodo_des.reset_index(inplace = True)
oot.reset_index(inplace=True)

print(periodo_des.shape, oot.shape)

(2506, 12) (94, 12)


In [10]:
#  Definindo x e y
x_oot = oot.drop(['result', 'date', 'index'], axis = 1)
y_oot = oot['result']
x = periodo_des.drop(['result', 'date', 'index'], axis = 1)
y = periodo_des['result']
x_oot.shape

(94, 9)

In [11]:
#  Separando a base de desenvolvimento em treino e teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 42)
x_train.shape, x_test.shape

((1879, 9), (627, 9))

In [12]:
#  Definindo o modelo de trabalho - Gradient Boosting

model = GradientBoostingClassifier()
model.fit(x_train, y_train)

#  Predict
# predict the mode
y_pred = model.predict(x_test)
classification_report(y_test, y_pred)
# performance evaluation metrics
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.91      0.88      0.90       337
           1       0.86      0.90      0.88       290

    accuracy                           0.89       627
   macro avg       0.89      0.89      0.89       627
weighted avg       0.89      0.89      0.89       627



In [13]:
#  Definindo parâmetros
param_grid = {'n_estimators': [10, 100, 300, 600] 
              , 'min_samples_leaf': [1, 10, 20, 50]
              , 'learning_rate': [.05, .1, .25, .5, 1]
             }

In [14]:
#  Separando os valores de cv
#  **parâmetro CV:** Significa *cross validation* (ou validação cruzada). 
#  Nele indicamos os índices das observações que serão utilizadas para desenvolver o modelo e para validar respectivamente. 
#  Através deste parâmetro, a função ainda consegue fazer outras modalidades de validação cruzada.

cv = KFold(n_splits=10, shuffle=True, random_state=10)

In [15]:
#  Fazendo a procura dos melhores parâmetros no modelo
grid_search = GridSearchCV(model,
                           param_grid=param_grid,
                          scoring = 'roc_auc',
                          cv = cv)

grid_search.fit(x, y)
print(grid_search.best_estimator_)

GradientBoostingClassifier(learning_rate=0.05, min_samples_leaf=50,
                           n_estimators=300)


In [19]:
#  Aplicar melhores resultados aqui
model = GradientBoostingClassifier(min_samples_leaf = 50, n_estimators = 300, learning_rate = 0.05)
model.fit(x_train, y_train)


# predict the mode
y_pred = model.predict(x_test)
av1 = classification_report(y_test, y_pred)
y_pred1 = model.predict(x_oot)
av2 = classification_report(y_oot, y_pred1)
# performance evaluatio metrics
print(av1, av2)

              precision    recall  f1-score   support

           0       0.89      0.92      0.90       324
           1       0.91      0.87      0.89       303

    accuracy                           0.90       627
   macro avg       0.90      0.90      0.90       627
weighted avg       0.90      0.90      0.90       627
               precision    recall  f1-score   support

           0       0.91      0.89      0.90        47
           1       0.90      0.91      0.91        47

    accuracy                           0.90        94
   macro avg       0.90      0.90      0.90        94
weighted avg       0.90      0.90      0.90        94



In [20]:
#  Avaliando modelo na base de treino
y_pred0 = model.predict(x_train)
av0 = classification_report(y_train, y_pred0)
print(av0)

              precision    recall  f1-score   support

           0       0.93      0.93      0.93       929
           1       0.93      0.94      0.94       950

    accuracy                           0.93      1879
   macro avg       0.93      0.93      0.93      1879
weighted avg       0.93      0.93      0.93      1879



In [21]:
#  Separando as features para identificação da importância das variáveis
feature_list = list(x.columns)
#  Importância das variáveis
importances = list(model.feature_importances_)
# Print out the feature and importances 
print(feature_list, importances)

['score', 'goals_solo', 'goals_assists', 'saves', 'shots', 'shooting_percentage', 'demos_inflicted', 'demos_taken', 'save_conceded'] [0.02564706298325827, 0.06831742141470526, 0.3383280505529951, 0.08135839456497004, 0.052512757495741205, 0.15478413650668604, 0.0021155870528686434, 0.00847099264998209, 0.2684655967787935]
